In [115]:
import pandas as pd
import re
import os
import zipfile
import json
import asyncio
from aio_pika import DeliveryMode, Message, connect
from config import conn

In [3]:
def create_proxy_extension(inp):
	PROXY_HOST, PROXY_PORT, PROXY_USER, PROXY_PASS=inp.split(':')
	manifest_json = """
	{
	"version": "1.0.0",
	"manifest_version": 2,
	"name": "Chrome Proxy",
	"permissions": [
		"proxy",
		"tabs",
		"unlimitedStorage",
		"storage",
		"<all_urls>",
		"webRequest",
		"webRequestBlocking"
	],
	"background": {
		"scripts": ["background.js"]
	},
	"minimum_chrome_version":"22.0.0"
	}
	"""

	background_js = """
	var config = {
		mode: "fixed_servers",
		rules: {
		singleProxy: {
			scheme: "http",
			host: "%s",
			port: parseInt(%s)
		},
		bypassList: ["localhost"]
		}
	};

	chrome.proxy.settings.set({value: config, scope: "regular"}, function() {});

	function callbackFn(details) {
		return {
			authCredentials: {
			username: "%s",
			password: "%s"
			}
		};
	}

	chrome.webRequest.onAuthRequired.addListener(
			callbackFn,
			{urls: ["<all_urls>"]},
			['blocking']
	);
	""" % (PROXY_HOST, PROXY_PORT, PROXY_USER, PROXY_PASS)
	with zipfile.ZipFile(os.path.join('proxy_extensions', PROXY_HOST+'.zip'), 'w') as zp:
		zp.writestr("manifest.json", manifest_json)
		zp.writestr("background.js", background_js)

#create_proxy_extension("185.42.27.255:10244:hhC6frMjKs:Wcnh3W2TPG")
create_proxy_extension("195.19.159.132:10244:QtnwmJ9MD4:cBM6nMQL95")
create_proxy_extension("195.19.159.131:10244:KT3rHKWExY:VwgtwMRdxW")
create_proxy_extension("195.19.159.99:10244:cYXGe2cAGu:ZkrFP8NVd7")
#create_proxy_extension("194.226.239.50:10244:v8HMdtkFrQ:6wZRUtDGMA")
create_proxy_extension("195.19.159.91:10244:W698HjfS66:x6sLLRm7DM")
#create_proxy_extension("83.171.240.137:10244:udV72GA6e3:p7hnrsArdu")
create_proxy_extension("5.8.16.115:10244:gtcxa3BaSk:aRSBUfMyWC")

In [5]:
df=pd.read_excel('gpb_adm_data.xlsx', sheet_name='TIDS')[['TID', 'longitude', 'latitude']].set_index('TID')

In [113]:
pd.DataFrame(columns=['TID', 'latitude', 'longitude', 'link', 'adress', 'metro_station', 'distance_to_station']
            ).to_csv('adm_parsing_data.csv', index=False)
pd.DataFrame(columns=['TID', 'organization_name', 'organization_name']
            ).to_csv('adm_organizations.csv', index=False)

In [21]:
async def listening():
    connection = await connect(conn)
    async with connection:
        channel = await connection.channel()
        queue = await channel.declare_queue("parsing_output", durable=True)
        async for msg in queue:
            if msg.body!=b'success':
                print(msg.body.decode())
                print('='*20)

t1=asyncio.create_task(listening())

('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


In [33]:
async def main() -> None:
    # Perform connection
    connection = await connect(conn)
    async with connection:
        # Creating a channel
        channel = await connection.channel()

        # Declaring queue
        queue = await channel.declare_queue("parsing_input", durable=True)
        # Sending the message
        for tids, r in df.iloc[1000:].iterrows():
            msg=json.dumps({'tids': tids, 'latitude': r['latitude'], 'longitude': r['longitude']})
            await channel.default_exchange.publish(Message(msg.encode(), delivery_mode=DeliveryMode.PERSISTENT),
                    routing_key=queue.name)
        #await channel.default_exchange.publish(
        #    Message(b"quit", delivery_mode=DeliveryMode.PERSISTENT),
        #    routing_key=queue.name,
        #)
        print('message has been send')

t2=asyncio.create_task(main())

message has been send
